In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
title = 'ShortSession8'

In [10]:
# Processed raw data: maintenance data excluded, nan value interpolated
mouse_pos = pd.read_parquet('mousepos.parquet', engine='pyarrow')
display(mouse_pos)

,x,y,filtered_position_x,filtered_position_y,filtered_velocity_x,filtered_velocity_y,filtered_acceleration_x,filtered_acceleration_y,smoothed_position_x,smoothed_position_y,smoothed_velocity_x,smoothed_velocity_y,smoothed_acceleration_x,smoothed_acceleration_y,smoothed_speed,smoothed_acceleration
time,,,,,,,,,,,,,,,,
2022-08-15 13:11:30.051296234,237.836426,531.666687,237.829709,531.657690,0.102785,0.587669,0.553673,1.543887,238.050332,531.960782,2.387246,7.745330,114.768281,359.236470,8.104880,377.124117
2022-08-15 13:11:30.071328163,238.111542,532.057739,237.982273,531.891373,1.593067,4.486184,42.815478,112.023049,238.127711,532.210361,5.350734,17.212558,148.174413,473.361412,18.025053,496.010769
2022-08-15 13:11:30.091296196,238.459061,532.976929,238.272732,532.662194,10.243183,31.701830,199.748503,618.333911,238.258487,532.633460,7.726831,25.097328,118.804853,394.238480,26.259851,411.750619
2022-08-15 13:11:30.111328125,238.480743,533.356445,238.491931,533.371544,13.379086,42.335027,185.857466,589.076910,238.425833,533.184975,9.007748,30.054202,64.045820,247.843695,31.375063,255.985086
2022-08-15 13:11:30.131296158,238.517136,533.782959,238.594819,533.916195,10.895791,40.155776,94.134080,359.237494,238.607230,533.811753,9.131985,32.623583,6.211870,128.469050,33.877593,128.619144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-15 16:01:44.127168178,243.603149,520.372925,243.550329,520.184904,-40.161556,95.126141,-169.686206,-682.827866,243.489848,520.506842,-46.811486,106.361583,-371.375069,-528.494617,116.207149,645.930338
2022-08-15 16:01:44.147136211,242.608215,522.574097,242.637712,522.424665,-45.871962,97.701467,-205.253499,-404.786309,242.466036,522.501959,-55.569678,93.150126,-437.909620,-660.572828,108.466286,792.541038
2022-08-15 16:01:44.167168140,241.278580,524.524597,241.391188,524.470203,-58.821244,95.514220,-341.035949,-303.577917,241.258971,524.198283,-65.136865,76.482293,-478.359344,-833.391658,100.460701,960.921077
